Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки 
Для оценки качества теггеров был собран корпус из отрывков стихотворений Игоря Северянина. В этом корпусе:
1. Много неологизмов, что может затруднить разметку для теггеров, опирающихся на словари. Также затруднит разметку то, что основа у неологизмов именная, а морфология глагольная: арфеет, осудьбить
2. Встречается частеречная омонимия: (я) жажду, настоящее, грядущее
3. Встречаются слова с дефисом, как допускающие, так и не допускающие разбиение: крем-брюле, отчего-то, трухло-злых, лилово-изнеженном
4. Встречаются сложные слова, у которых вторая часть совпадает со словами или вторыми частями сложных слов: полпорции
5. Встречаются слова с «неизвестными» корнями, не содержащие продуктивных аффиксов, для которых носители языка могут однозначно определить часть речи: буше, вирелэ
акой тегсет вы берёте для разметки и почему.

Какой тегсет вы берёте для разметки и почему.
Поскольку славистика в целом и русская морфология в частности не входят в область моих интересов, я разметила корпус с точки зрения "наивного носителя": noun - существительные, в т.ч. имена собственные; adj - краткие и полные прилагательные, в т.ч. степени сравнения; adv - наречия; v - все формы глаголов, в т.ч. причастия и деепричастия; num - порядковые и количественные числительные; rest - всё остальное.

In [62]:
with open ('Severyanin.txt', 'r', encoding='utf8') as file:
    text = file.read()

In [63]:
import xlrd

In [104]:
# собираем словарь ручной разметки "словоформа: POS-тег"
file = xlrd.open_workbook('POS_human.xls')
sheet = file.sheet_by_name('POS')
human_tags = {}
l = sheet.nrows
for i in range(l):
    human_tags[sheet.cell_value(i, 0).lower()] = sheet.cell_value(i, 1)

In [146]:
len(human_tags) # число различных словоформ

163

Не вижу смысла писать сложный конвертер тегов, когда можно обойтись тремя небольшими словарями "тег ручной разметки: список тегов анализатора"

pymystem3 POS tagset
A прилагательное
ADV наречие
ADVPRO местоименное наречие
ANUM числительное-прилагательное
APRO местоимение-прилагательное
COM часть композита - сложного слова
CONJ союз
INTJ междометие
NUM числительное
PART частица
PR предлог
S существительное
SPRO местоимение-существительное
V глагол

In [129]:
mystem_to_human = {'noun':['COM','S'], 'adj':['A'], 'adv':['ADV'], 'v':['V'], 'num':['ANUM','NUM'], 'rest':['ADVPRO','APRO','COM','CONJ', 'INTJ','PART','PR','SPRO']}

In [3]:
from pymystem3 import Mystem
mystem = Mystem()

In [134]:
# лемматизируем текст и собираем словарь разметки "словоформа: POS-тег" для pymystem3
mystem_lemmas = mystem.analyze(text)
mystem_pos = {}
for word in mystem_lemmas:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_pos[word['text'].lower()] = [pos]

Pymorphy POS tagset - OpencorporaTag
NOUN 	имя существительное 	хомяк
ADJF 	имя прилагательное (полное) 	хороший
ADJS 	имя прилагательное (краткое) 	хорош
COMP 	компаратив 	лучше, получше, выше
VERB 	глагол (личная форма) 	говорю, говорит, говорил
INFN 	глагол (инфинитив) 	говорить, сказать
PRTF 	причастие (полное) 	прочитавший, прочитанная
PRTS 	причастие (краткое) 	прочитана
GRND 	деепричастие 	прочитав, рассказывая
NUMR 	числительное 	три, пятьдесят
ADVB 	наречие 	круто
NPRO 	местоимение-существительное 	он
PRED 	предикатив 	некогда
PREP 	предлог 	в
CONJ 	союз 	и
PRCL 	частица 	бы, же, лишь
INTJ 	междометие

In [17]:
pymorphy_to_human = {'noun':['NOUN'], 'adj':['ADJF', 'ADJS', 'COMP'], 'adv':['ADVB'], 'v':['VERB','INFN','PRTF','PRTS','GRND'], 'num':['NUMR'], 'rest':['NPRO','PRED','PREP','CONJ', 'INTJ','PRCL']}

In [14]:
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [26]:
# pymorphy2 не умеет работать с целым текстом, приходится разбивать текст на словоформы, убирая переносы строк и пунктуацию
lines = text.split('\n')
sents = []
for line in lines:
    sent = [word.strip(punctuation) for word in line.split()] #remove punctuation
    sents.extend(sent)

In [140]:
# лемматизируем текст и собираем словарь разметки "словоформа: POS-тег" для pymorphy2
pymorphy_lemmas = [morph.parse(w) for w in sents]
pymorphy_pos = {}
for lemma in pymorphy_lemmas:
    key = lemma[0].word
    value = []
    for parse in lemma:
        value.append(parse.tag.POS)
    pymorphy_pos[key.lower()] = value

SpaCY -  Universal POS tags
    ADJ: adjective
    ADP: adposition
    ADV: adverb
    AUX: auxiliary
    CCONJ: coordinating conjunction
    DET: determiner
    INTJ: interjection
    NOUN: noun
    NUM: numeral
    PART: particle
    PRON: pronoun
    PROPN: proper noun
    PUNCT: punctuation
    SCONJ: subordinating conjunction
    SYM: symbol
    VERB: verb
    X: other

In [73]:
spacy_to_human = {'noun':['NOUN','PROPN'], 'adj':['ADJ'], 'adv':['ADV'], 'v':['AUX', 'VERB'], 'num':['NUM'], 'rest':['ADP','CCONJ','DET','PART', 'INTJ','PRON','SCONJ','X']}

In [49]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [107]:
# лемматизируем текст и собираем словарь разметки "словоформа: POS-тег" для spacy
doc = nlp(text)
spacy_pos = {}
for token in doc:
    if token.pos_ not in ('PUNCT','SPACE'):
        spacy_pos[token.text.lower()] = [token.pos_]

In [147]:
# функция для определения качества разметки
def accuracy(tag, pos):
    true_positive = 0
    wrong_split = []
    wrong_pos = []
    for key in pos:
        if key not in human_tags.keys(): # выявляем разбитые по дефису словоформы
            wrong_split.append(key)
        else:
            human_pos = human_tags[key]
            if set(pos[key]) & set(tag[human_pos]): # если хотя бы один тег анализатора совпал с тегом ручной разметки, засчитываем попадание
                true_positive += 1
            else:
                wrong_pos.append((key, pos[key])) # если не совпал ни один, сохраняем ошибочную разметку. Посмотрим, где именно теггер ошибся
    return(true_positive/163, wrong_split, wrong_pos) # 163 - количество уникальных словоформ в тексте


In [148]:
accuracy(spacy_to_human, spacy_pos)

(0.852760736196319,
 ['трухло',
  '-',
  'злых',
  'крем',
  'брюле',
  'лилово',
  'изнеженном',
  'отчего',
  'то'],
 [('бесстыжей', ['NOUN']),
  ('тьмой', ['ADJ']),
  ('надо', ['ADV']),
  ('можно', ['ADV']),
  ('пора', ['NOUN']),
  ('изыски', ['ADV']),
  ('муаровом', ['NOUN']),
  ('олуненной', ['ADJ']),
  ('морево', ['NOUN']),
  ('лазорева', ['PROPN']),
  ('разузорена', ['NOUN']),
  ('точно', ['ADV']),
  ('лапы', ['ADJ']),
  ('утонченной', ['ADJ']),
  ('новобрачная', ['ADJ']),
  ('тишь', ['VERB']),
  ('куда', ['ADV']),
  ('былом', ['ADJ']),
  ('грядущем', ['ADJ']),
  ('настоящем', ['ADJ'])])

Разбил все слова по дефису. Ошибся в тегах частотных существительных: тьма, изыски, лапы, тишь. Ошибся в тегах прилагательных с очевидными морфологическими признаками: бесстыжий, муаровый. Сделал из краткого прилагательного фамилию: лазорева. Не справился со сложными случаями частеречной омонимии: новобрачная, былое, грядущее, настоящее, точно, пора; жажду определил верно, как глагол. Не отличил прилагательное от страдательного причастия: утонченный, олуненный. 
Не очень-то spacy понимает в русской поэзии Серебряного века :)

In [149]:
accuracy(mystem_to_human, mystem_pos)

(0.9079754601226994,
 ['трухло', 'злых', 'лилово', 'изнеженном'],
 [('осудьбить', ['S']),
  ('надо', ['ADV']),
  ('можно', ['ADV']),
  ('пора', ['ADV']),
  ('зальдись', ['ADV']),
  ('морево', ['S']),
  ('лазорева', ['S']),
  ('утонченной', ['A']),
  ('всегда', ['ADVPRO']),
  ('арфеет', ['S']),
  ('былом', ['A']),
  ('настоящем', ['A']),
  ('жажду', ['S'])])

Разбил по дефису словоформы, для которых это вполне оправдано: трузло-злой, лилово-изнеженный. Отчего-то и крем-брюле остались целыми и с правильными тегами. Неправильно определил неологизмы, где именной корень имеет глагольную морфологию: осудьбить, арфеет, зальдись. Также сделал из краткого прилагательного фамилию: лазорева. Не справился с частеречной омонимией: жажду, былое, настоящее; грядущее и новобрачную определил верно.
Это победитель соревнования.

In [150]:
accuracy(pymorphy_to_human, pymorphy_pos)

(0.901840490797546,
 ['всё', 'придётся', '—', 'утончённой', 'её', 'поёт'],
 [('весь', ['ADJF', 'ADJF', 'NOUN', 'NOUN', 'VERB']),
  ('твоих', ['ADJF', 'ADJF', 'ADJF']),
  ('надо', ['PRED', 'PREP']),
  ('можно', ['PRED']),
  ('морево',
   ['NOUN',
    'NOUN',
    'NOUN',
    'NOUN',
    'ADJS',
    'ADJF',
    'ADJF',
    'ADJS',
    'NOUN',
    'NOUN',
    'NOUN',
    'NOUN',
    'ADJF',
    'ADJF']),
  ('ваше', ['ADJF', 'ADJF']),
  ('ваша', ['ADJF']),
  ('новобрачная', ['ADJF']),
  ('такая', ['ADJF', 'GRND']),
  ('былом', ['ADJF', 'ADJF']),
  ('отчего-то', ['ADVB'])])

Дописал точки над Ё - это очень неудобно, поскольку такое поведение анализатора сложно предвидеть и учесть. Словоформы по дефису не разбиваются независимо от структуры. Хотя ошибок в тегах у этого анализатора меньше, нужно принять во внимание, что каждой словоформе приписывалось несколько тегов, поэтому у pymorphy была большая фора. И даже с ней он не сумел обойти mystem по accuracy.